In [ ]:
from pathlib import Path
from sklearn.mixture import GaussianMixture
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Configure notebook and get raw data

In [ ]:
data_path = Path("../../data/MACH_data/data.cleaned.csv")
codebook_path = Path("../../data/MACH_data/codebook.txt")
original_data_df = pd.read_csv(data_path)

# Choose features
Sticking to just the actual question response for now.

In [ ]:
question_responses = ["Q1A", "Q2A", "Q3A", "Q4A", "Q5A", "Q6A", "Q7A", "Q8A", "Q9A", "Q10A", 
                      "Q11A", "Q12A", "Q13A", "Q14A", "Q15A", "Q16A", "Q17A", "Q18A", "Q19A", "Q20A"]
X = original_data_df[question_responses].copy()
print(f"Using features: {question_responses}  |  Shape: {X.shape}")
display(X.head())

# Data preprocessing
Scaling this data is likely not necessary since the data is usually between 1 and 5, with some NA values.

In [ ]:
X_clean = X.dropna().copy()
X_clean[question_responses] = X_clean[question_responses].astype(int)
Xs = pd.DataFrame(X_clean, columns=question_responses, index=X_clean.index)
# use only 5000 random datapoints
Xs_sample = Xs.sample(n=5000, random_state=42)
print(f"After preprocessing shape: {Xs_sample.shape}")
display(Xs_sample.head())
display(Xs_sample.describe().T.round(3))

# Cluster

In [ ]:
k_range = range(2, 7)
for k in k_range:
    gmm = GaussianMixture(n_components=k, random_state=0)
    clusters = gmm.fit_predict(Xs_sample)

    # get silhoutte score
    sil = silhouette_score(Xs_sample, clusters)
    print(f"Silhouette Score for k={k}: {sil}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

bic = []
aic = []
log_likelihood = []

Ks = range(1, 9)

for k in Ks:
    gmm = GaussianMixture(n_components=k, random_state=0)
    gmm.fit(Xs_sample)
    bic.append(gmm.bic(Xs_sample))
    aic.append(gmm.aic(Xs_sample))
    log_likelihood.append(gmm.score(Xs_sample))  # avg log-likelihood

# --- Normalize each metric using min-max ---
bic_norm = (np.array(bic) - np.min(bic)) / (np.max(bic) - np.min(bic))
aic_norm = (np.array(aic) - np.min(aic)) / (np.max(aic) - np.min(aic))
ll_norm  = (np.array(log_likelihood) - np.min(log_likelihood)) / (np.max(log_likelihood) - np.min(log_likelihood))

# --- Plot all on one chart ---
plt.plot(Ks, bic_norm, marker='o', label="BIC (normalized)")
plt.plot(Ks, aic_norm, marker='o', label="AIC (normalized)")
plt.plot(Ks, ll_norm, marker='o', label="Log-Likelihood (normalized)")

plt.xlabel("Number of Components (k)")
plt.ylabel("Normalized Value")
plt.title("BIC, AIC, and Log-Likelihood (Normalized)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(Xs_sample)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, k in enumerate([2, 4, 6]):
    gmm = GaussianMixture(n_components=k, random_state=0)
    labels = gmm.fit_predict(Xs_sample)
    score = silhouette_score(Xs_sample, labels)
    axes[i].scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="tab10", s=15)
    axes[i].set_title(f"PCA Projection ({k} clusters)\nSilhouette: {score:.3f}")
    axes[i].set_xlabel("PCA 1")
    axes[i].set_ylabel("PCA 2")
plt.tight_layout()
plt.show()

# Analyze cluster mean responses

In [ ]:
gmm = GaussianMixture(n_components=6, random_state=0)
labels = gmm.fit_predict(Xs_sample)
df = Xs_sample.copy()
df["Cluster"] = labels

cluster_means = df.groupby("Cluster").mean()
display(cluster_means)

plt.figure(figsize=(10, 6))
sns.heatmap(cluster_means, annot=True, cmap="coolwarm", cbar_kws={"label": "Mean Response"})
plt.title("Mean MACH-IV Question Responses per Cluster")
plt.xlabel("Question")
plt.ylabel("Cluster")
plt.show()

In [ ]:
cluster_modes = df.groupby("Cluster")[question_responses] \
                  .agg(lambda x: x.mode().iloc[0]) # first mode
display(cluster_modes)

plt.figure(figsize=(10, 6))
sns.heatmap(cluster_modes, annot=True, cmap="coolwarm", cbar_kws={"label": "Median Response"})
plt.title("Mode MACH-IV Question Responses per Cluster")
plt.xlabel("Question")
plt.ylabel("Cluster")
plt.show()

# Check for patterns with other columns

In [ ]:
other_responses = ["TIPI1","TIPI2","TIPI3","TIPI4","TIPI5","TIPI6","TIPI7","TIPI8","TIPI9","TIPI10",
                   "VCL1","VCL2","VCL3","VCL4","VCL5","VCL6","VCL7","VCL8","VCL9","VCL10", "VCL11","VCL12","VCL13","VCL14","VCL15", "VCL16",
                   "education", "urban", "gender", "engnat", "age", "hand", "religion", "orientation", "race", "voted", "married", "familysize", "major"]

# remove major for now
other_responses.remove("major")

In [ ]:
gmm = GaussianMixture(n_components=6, random_state=0)
labels = gmm.fit_predict(Xs_sample)
Xs_sample_with_other_responses = original_data_df.loc[Xs_sample.index]
df = pd.DataFrame(Xs_sample_with_other_responses, columns=other_responses, index=Xs_sample_with_other_responses.index)
df["Cluster"] = labels

scaler = MinMaxScaler()
normalized_df = pd.DataFrame(
    scaler.fit_transform(df),
    columns=df.columns,
    index=df.index
)

cluster_means = normalized_df.dropna().groupby("Cluster").mean()
display(cluster_means)

plt.figure(figsize=(20, 6))
sns.heatmap(cluster_means, annot=True, cmap="coolwarm", cbar_kws={"label": "Mean Response"})
plt.title("Mean MACH-IV Question Responses per Cluster")
plt.xlabel("Question")
plt.ylabel("Cluster")
plt.show()

In [ ]:
cluster_modes = normalized_df.groupby("Cluster")[other_responses] \
                  .agg(lambda x: x.mode().iloc[0]) # first mode
display(cluster_modes)

plt.figure(figsize=(20, 6))
sns.heatmap(cluster_modes, annot=True, cmap="coolwarm", cbar_kws={"label": "Median Response"})
plt.title("Mode MACH-IV Question Responses per Cluster")
plt.xlabel("Question")
plt.ylabel("Cluster")
plt.show()